In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [10]:
model = read_sbml_model('data/models/iMM904.xml')
model_new = model

In [11]:
# Reaction 1 (HA-TAL):L-tyrosine <=> (E).4.coumarate +   NH4+
new_reaction1 = Reaction('HA-TAL') # 3-(o)xopropanoate:(N)ADP+ (o)xidoreductase
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction1.add_metabolites({model_new.metabolites.tyr__L_c: -1, # Search L-tyrisine ID
                               coumarate4: 1, # 3-Oxopropanoate
                               model_new.metabolites.nh4_c: 1, # Search fpr NH4 ID
                              })
model_new.add_reactions([new_reaction1])

In [12]:
new_reaction2 = Reaction('AT-4CL1')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2.add_metabolites({model_new.metabolites.coumarate4: -1,
                               model_new.metabolites.atp_c: -1,
                               model_new.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_new.metabolites.amp_c: 1,
                               model_new.metabolites.ppi_c: 1,
                              })
model_new.add_reactions([new_reaction2])

In [13]:
new_reaction3 = Reaction('VVVST1')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3.add_metabolites({model_new.metabolites.e4_coumaroyl_coa: -1,
                               model_new.metabolites.h_c: -3,
                               model_new.metabolites.malcoa_c: -3,
                               model_new.metabolites.co2_c: 4,
                               model_new.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_new.add_reactions([new_reaction3])

In [14]:
model_new.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x07fca371b5a90
Stoichiometry,e4_coumaroyl_coa + 3 h_c + 3 malcoa_c --> 4 co2_c + 4 coa_c + trans_resv + 3 H+ + 3 Malonyl CoA C24H33N7O19P3S --> 4 CO2 CO2 + 4 Coenzyme A +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [18]:
from cobra.io import write_sbml_model
write_sbml_model(model_new,'data/models/yeast_resv.xml')

#### how much resveratrol is produced?

In [16]:
model_new.add_boundary(model_new.metabolites.trans_resv, type='demand')

Reaction identifier,DM_trans_resv
Name,demand
Memory address,0x07fca371beba8
Stoichiometry,trans_resv --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [17]:
with model_new:
    model_new.objective=model_new.reactions.VVVST1
    lets_produce_resv= model_new.optimize().objective_value
lets_produce_resv

1.1785714285714293